In [2]:
!pip freeze

absl-py==1.0.0
aiohttp==3.8.1
aiosignal==1.2.0
anyio==3.6.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
async-timeout==4.0.1
attrs==21.2.0
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.0
black==22.6.0
bleach==5.0.1
cachetools==4.2.4
certifi==2021.10.8
cffi==1.15.1
charset-normalizer==2.0.7
click==8.0.3
coloredlogs==15.0.1
cycler==0.11.0
datasets==1.13.3
debugpy==1.6.2
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.4
entrypoints==0.4
executing==0.9.1
fastjsonschema==2.16.1
filelock==3.4.0
Flask==2.0.2
Flask-API==3.0.post1
flatbuffers==1.12
fonttools==4.34.4
frozenlist==1.2.0
fsspec==2021.11.0
future==0.18.2
gast==0.4.0
google-auth==2.3.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
greenlet==1.1.2
grpcio==1.41.1
h5py==3.7.0
huggingface-hub==0.8.1
humanfriendly==10.0
idna==3.3
importlib-metadata==4.8.2
ipykernel==6.15.1
ipython==8.4.0
ipython-genutils==0.2.0
itsdangerous==2.0.1
jedi==0.18.1
Jinja2==3.0.3
joblib==1.1.0
JPype1==1.4.0
json5==0.9.

In [10]:
import pandas as pd
import numpy as np
import os
# from konlpy import Mecab

!pwd

/root/2022_chatbot/assets


In [44]:
WORK_DIR="/root/2022_chatbot/assets"
MECAB_DIR="/root/2022_chatbot/mecab-ko-dic-2.1.1-20180720"

%cd $WORK_DIR
!ls

/root/2022_chatbot/assets
mecab_user_dic.ipynb  tmp


In [7]:
# 미리 만들어둔 사용자 사전 nnp파일 read
with open(WORK_DIR+"/nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '모바일서비스팀,,,,NNP,*,F,모바일서비스팀,*,*,*,*,*']

In [32]:
'메,롱'[-1]

'롱'

## Sample - 계급 추가

In [41]:
def is_hangul(word):
    for w in word:
        if 44032 > ord(w) or ord(w) > 55203:
            if w != ' ':
                return False
    
    return True

def has_coda(word):
    return (ord(word[-1])-44032) %28 == 0

# 종성유무 반환 함수
has_coda('대령')
is_hangul('메 롱')

True

In [11]:
sample_df = pd.read_csv(WORK_DIR+'/tmp/계급.csv')
sample_df

,0
0,소장
1,준장
2,대령
3,중령
4,소령
5,대위
6,중위
7,소위
8,준위
9,원사


In [17]:
with open(WORK_DIR+"/nnp.csv", 'w', encoding='utf-8') as f:
    for w in sample_df['0']:
        coda = 'F' if has_coda(w) else 'T'
        line = f'{w},,,,NNP,*,{coda},{w},*,*,*,*\n'
        f.write(line)

%cat nnp.csv

소장,,,,NNP,*,T,소장,*,*,*,*
준장,,,,NNP,*,T,준장,*,*,*,*
대령,,,,NNP,*,T,대령,*,*,*,*
중령,,,,NNP,*,T,중령,*,*,*,*
소령,,,,NNP,*,T,소령,*,*,*,*
대위,,,,NNP,*,F,대위,*,*,*,*
중위,,,,NNP,*,F,중위,*,*,*,*
소위,,,,NNP,*,F,소위,*,*,*,*
준위,,,,NNP,*,F,준위,*,*,*,*
원사,,,,NNP,*,F,원사,*,*,*,*
상사,,,,NNP,*,F,상사,*,*,*,*
중사,,,,NNP,*,F,중사,*,*,*,*
하사,,,,NNP,*,F,하사,*,*,*,*
병장,,,,NNP,*,T,병장,*,*,*,*
상병,,,,NNP,*,T,상병,*,*,*,*
일병,,,,NNP,*,T,일병,*,*,*,*
이병,,,,NNP,*,T,이병,*,*,*,*


## nnp 단어추가

In [43]:
with open(WORK_DIR+"/nnp.csv", 'w', encoding='utf-8') as f:
    for dirname, _, filenames in os.walk(WORK_DIR+'/tmp'):
        for filename in filenames:
            filepath = os.path.join(dirname, filename)
            sample_df = pd.read_csv(filepath)
            
            fnm = filename.split(".")[0]
            print(f'[{fnm}] # of data : {len(sample_df.index)} ===================>')
            tag = 'NNP'
            nm = '*'
            
            if fnm=='저자':
                nm = '인명'
            elif fnm=='지역':
                nm = '지명'
            
            for w in sample_df['0']:
                try:
                    if is_hangul(w) == True:
                        coda = 'F' if has_coda(w) else 'T'
                        line = f'{w},,,,NNP,{nm},{coda},{w},*,*,*,*,*\n'
                        f.write(line)  
                except:
                    continue

[군구분] # of data : 4 ===================>
[지역] # of data : 284 ===================>
[저자] # of data : 68741 ===================>
[시설] # of data : 188 ===================>
[주택] # of data : 2093 ===================>
[부대번호] # of data : 169 ===================>
[부대] # of data : 18071 ===================>
[직급] # of data : 23 ===================>
[계급] # of data : 17 ===================>
[식단] # of data : 18 ===================>
[도서명] # of data : 118643 ===================>


## 우선순위 단어 조정

In [46]:
%cd $MECAB_DIR

/root/2022_chatbot/mecab-ko-dic-2.1.1-20180720


In [52]:
with open("user-nnp.csv", 'r', encoding='utf-8') as f:
    userdic = f.readlines()

In [54]:
userdic[:10]

['육군,1786,3546,2456,NNP,*,T,육군,*,*,*,*,*\n',
 '해군,1786,3546,2568,NNP,*,T,해군,*,*,*,*,*\n',
 '해병대,1786,3545,2937,NNP,*,F,해병대,*,*,*,*,*\n',
 '공군,1786,3546,2411,NNP,*,T,공군,*,*,*,*,*\n',
 '전라남도,1789,3552,5497,NNP,지명,F,전라남도,*,*,*,*,*\n',
 '달서구,1789,3552,5497,NNP,지명,F,달서구,*,*,*,*,*\n',
 '강북구,1789,3552,5497,NNP,지명,F,강북구,*,*,*,*,*\n',
 '태안군,1789,3553,5497,NNP,지명,T,태안군,*,*,*,*,*\n',
 '안양시,1789,3552,4268,NNP,지명,F,안양시,*,*,*,*,*\n',
 '계양구,1789,3552,5497,NNP,지명,F,계양구,*,*,*,*,*\n']

In [59]:
sample_line = userdic[0]
arr = sample_line.split(',')

In [62]:
arr[3] = '0'
ret = ','.join(arr)
ret

'육군,1786,3546,0,NNP,*,T,육군,*,*,*,*,*\n'

In [64]:
with open("user_nnp_new.csv", 'w', encoding='utf-8') as f:
    for line in userdic:
        arr = line.split(',')    
        arr[3] = '0'
        f.write(','.join(arr))